In [1]:
import mysql.connector

conn = mysql.connector.connect(user='root',password='jagadeesh',host='localhost')
print(conn)

In [47]:
curs = conn.cursor()

In [14]:
curs

# 1. Implement Inventory Management in Python with MySQL

a) Inventory management, a critical element of the supply chain, is the tracking of inventory from manufacturers to warehouses and from these facilities to a point of sale. The goal of inventory management is to have the right products in the right place at the right time.

b) The required Database is Inventory, and the required Tables are Purchases, Sales and Inventory

c) Note: Apply your thoughts to demonstrate the DB Operation in Python.

In [6]:
#create database
sql='create database Inventory'
curs.execute(sql)

In [15]:
curs.execute('show databases')

In [16]:
for i in curs:
    print(i)

('company',)
('information_schema',)
('innodb',)
('inventory',)
('mypractise',)
('mysql',)
('performance_schema',)
('sakila',)
('shirts_db',)
('sys',)
('world',)


In [17]:
curs.execute("use Inventory")

In [25]:
curs.execute("""CREATE TABLE Purchases (
    purchase_id INT AUTO_INCREMENT PRIMARY KEY,
    product_name VARCHAR(255),
    quantity INT,
    cost_price DECIMAL(10, 2))""")

In [26]:
curs.execute("""CREATE TABLE Sales (
    sale_id INT AUTO_INCREMENT PRIMARY KEY,
    product_name VARCHAR(255),
    quantity INT,
    selling_price DECIMAL(10, 2))""")

In [28]:
curs.execute("""CREATE TABLE Inventory (
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    product_name VARCHAR(255),
    quantity INT,
    cost_price DECIMAL(10, 2),
    selling_price DECIMAL(10, 2))""")

# 2. Customer Order Processing

Scenario: You are building a customer order processing system for an e-commerce company. The system needs to interact with a MySQL database to store customer orders, products, and order details.

1. Design a MySQL database schema for the order processing system, including tables for

customers, products, and orders. 2. Write a Python program that connects to the database and allows customers to place

new orders. 3. Implement a feature that calculates the total cost of an order and updates product

quantities in the database.

4. How would you handle cases where a product is no longer available when a customer places an order?

5. Develop a function to generate order reports for the company's finance department.


In [29]:
sql='create database orderprocess'
curs.execute(sql)

In [60]:
curs.execute("use orderprocess")

In [34]:
curs.execute("""CREATE TABLE Customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(30) NOT NULL,
    email VARCHAR(30) NOT NULL,
    address varchar(255))""")

In [35]:
curs.execute("""CREATE TABLE Products (
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    price INT,
    quantity_in_stock INT)""")

In [36]:
curs.execute("""CREATE TABLE Orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_cost INT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id))""")

In [37]:
curs.execute("""CREATE TABLE OrderDetails (
    order_detail_id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity_ordered INT,
    unit_price INT,
    total_price INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id))""")

In [38]:
curs.execute('show tables')

In [39]:
for i in curs:
    print(i)

('customers',)
('orderdetails',)
('orders',)
('products',)


In [52]:
insert="insert into Customers(name,email,address) values(%s,%s,%s)"
val=[('jagadeesh','j@gmail.com','vizag'),
     ('hari','h@gmail.com','kurnool'),
     ('arun','a@gmail.com','bangalore')
    ]
curs=conn.cursor()
curs.executemany(insert,val)

conn.commit()

print(curs.rowcount,'is inserted now')

3 is inserted now


In [61]:
insert1="insert into Products(name,price,quantity_in_stock) values(%s,%s,%s)"
val1=[('bag',500,5),
     ('bat',1500,5),
     ('ball',200,5)
    ]
curs=conn.cursor()
curs.executemany(insert1,val1)

conn.commit()
curs.close()

print(curs.rowcount,'is inserted now')

3 is inserted now


In [62]:
insert_query = "INSERT INTO Orders (customer_id, order_date, total_cost) VALUES (%s, %s, %s)"

order_data = [
    (1, '2023-09-15', 500),
    (2, '2023-09-16', 1200),
    (3, '2023-09-17', 750)
]

curs = conn.cursor()

curs.executemany(insert_query, order_data)

conn.commit()
curs.close()

print(curs.rowcount, 'rows inserted into Orders table')

3 rows inserted into Orders table


In [66]:
import mysql.connector

insert_query = "INSERT INTO OrderDetails (order_id, product_id, quantity_ordered, unit_price, total_price) VALUES (%s, %s, %s, %s, %s)"

order_details_data = [
    (1, 1, 3, 150, 450),
    (2, 2, 2, 25, 50),
    (3, 3, 5, 10, 50)
]


curs = conn.cursor()

curs.executemany(insert_query, order_details_data)

conn.commit()
curs.close()

print(curs.rowcount, 'rows inserted into OrderDetails table')


3 rows inserted into OrderDetails table


In [69]:
def place_order(customer_id, product_id, quantity_ordered):

    curs = conn.cursor()

    curs.execute("SELECT price, quantity_in_stock FROM Products WHERE product_id = %s", (product_id,))
    result = curs.fetchone()

    if result is None:
        print("Product not found.")
    else:
        price, quantity_in_stock = result
        if quantity_ordered > quantity_in_stock:
            print("Product not available in the required quantity.")
        else:
            total_price = price * quantity_ordered

            curs.execute("INSERT INTO Orders (customer_id, order_date, total_cost) VALUES (%s, CURDATE(), %s)", (customer_id, total_price))
            order_id = curs.lastrowid
            curs.execute("INSERT INTO OrderDetails (order_id, product_id, quantity_ordered, unit_price, total_price) VALUES (%s, %s, %s, %s, %s)", (order_id, product_id, quantity_ordered, price, total_price))

            curs.execute("UPDATE Products SET quantity_in_stock = quantity_in_stock - %s WHERE product_id = %s", (quantity_ordered, product_id))

            conn.commit()
            curs.close()
            print("Order placed successfully.")

place_order(1, 1, 2)

Order placed successfully.


In [70]:
place_order(2,2,10)

Product not available in the required quantity.


In [73]:
import csv

def generate_order_report(output_file):
    curs = conn.cursor()

    curs.execute("SELECT order_id, customer_id, order_date, total_cost FROM Orders")

    orders = curs.fetchall()

    # Write the results to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ["Order ID", "Customer ID", "Order Date", "Total Cost"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for order in orders:
            writer.writerow({
                "Order ID": order[0],
                "Customer ID": order[1],
                "Order Date": order[2],
                "Total Cost": order[3]
            })

    curs.close()

generate_order_report("order_report.csv")


# 3. You are tasked with developing a Python program that connects to a MySQL database, retrieves data from a table, performs some operations on the data, and updates the database with the modified data. Please write Python code to accomplish this task.

Instructions:

1. Assume that you have a MySQL database server running with the following details:

i. Host: localhost ii. Port: 3306

iii. Username: your username

iv. Password: your password

v. Database Name: your database vi. Table Name: your table

vil. The table has the following columns: id (int), name (varchar).

quantity (int).

2. Your Python program should;

1. Connect to the MySQL database.

ii. Retrieve all records from the your table table.

iii. Calculate the total quantity of all records retrieved.

iv. Update the quantity column of each record by doubling its value. v. Commit the changes to the database.

vi. Close the database connection.

3. Handle any potential errors that may occur during the database connection data manipulation, such as connection failures or SQL errors.

4. Provide comments in your code to explain each step.

In [25]:
import mysql.connector
conn = mysql.connector.connect(user='root', password='jagadeesh',host='localhost', database='Inventory') 
print(conn)

In [26]:
curs= conn.cursor()

In [4]:
curs.execute("use Inventory")

In [10]:
curs.execute("""CREATE TABLE Items ( 
id INT AUTO_INCREMENT PRIMARY KEY,
name VARCHAR(30),
quantity INT)""")

In [12]:
insert_query = "INSERT INTO Items (name, quantity) VALUES (%s, %s)"

insert_data = [('item1',10),('item2',20),('item3',30)]

curs.executemany (insert_query, insert_data)

conn.commit()

print(f" {curs.rowcount} rows inserted into Items.")

 3 rows inserted into Items.


In [27]:
select_query = "SELECT id, name, quantity FROM Items"

curs.execute(select_query)

records = curs.fetchall()

total_quantity=sum(record[2] for record in records)

update_query = "UPDATE Items SET quantity = quantity * 2" 

curs.execute(update_query)

conn.commit()

In [28]:
curs.execute("Select * from Items")

for i in curs:
    print(i)

(1, 'item1', 20)
(2, 'item2', 40)
(3, 'item3', 60)


# 4. You are developing an employee management system for a company. The database should store employee information, including name, salary, department, and hire date. Managers should be able to view and update employee details.

1. Design the database schema for the employee management system.

2. Write Python code to connect to the database and retrieve a list of employees in a specific department.

3. Implement a feature to update an employee's salary.

In [29]:
curs.execute("CREATE DATABASE EmployeeManagement;")

In [30]:
curs.execute("USE EmployeeManagement;")

In [31]:
curs.execute("""CREATE TABLE Employees (
    employee_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    salary DECIMAL(10, 2) NOT NULL,
    department VARCHAR(255) NOT NULL,
    hire_date DATE NOT NULL
)""")

In [36]:
insert_query = "INSERT INTO Employees (name, salary, department, hire_date) VALUES (%s, %s, %s, %s)"
employee_data = [("Jagadeesh", 55000.00, "HR", "2023-09-15"),
                ("hari", 60000.00, "Finance", "2023-09-16"),
                ("arun", 50000.00, "IT", "2023-09-17")]
curs.executemany(insert_query, employee_data)

In [38]:
def get_employees_in_department(department):
    query = "SELECT * FROM Employees WHERE department = %s"
    curs.execute(query, (department,))

    employees = curs.fetchall()

    for employee in employees:
        print(f"Employee ID: {employee[0]}, Name: {employee[1]}, Salary: {employee[2]}, Department: {employee[3]}, Hire Date: {employee[4]}")

In [39]:
def update_employee_salary(employee_id, new_salary):
    update_query = "UPDATE Employees SET salary = %s WHERE employee_id = %s"
    curs.execute(update_query, (new_salary, employee_id))
    conn.commit()

In [40]:
get_employees_in_department("HR")

Employee ID: 1, Name: Jagadeesh, Salary: 55000.00, Department: HR, Hire Date: 2023-09-15


In [41]:
update_employee_salary(1, 60000.00)

In [42]:
get_employees_in_department("HR")

Employee ID: 1, Name: Jagadeesh, Salary: 60000.00, Department: HR, Hire Date: 2023-09-15


# 1 Scenario: You are developing a complex Python project with multiple modules. To manage the project effectively, you need to import and use various modules. Additionally, you want to organize your code using namespaces and avoid naming conflicts.

Design a Python program that demonstrates the following:

1. Import multiple modules within your project.
2. Use the import statement to access functions, classes, and variables from imported modules.
3. Create your custom module and use it in your main program.
4. Handle naming conflicts and ensure proper namespacing.
5. Implement error handling for missing modules or incorrect module usage.

In [1]:
def say_hello():
    print("Hello from module1")

m1_var = "Variable from module1"

In [2]:
def print_variable(variable):
    print(f"Variable from module2: {variable}")

m2_variable = "I am from module2"

In [3]:
try:
    import m1
    import m2
    m1.say_hello()
    m2.print_variable(m2.m2_variable)
    
except ImportError as e:
    print(f"Error importing module: {e}")
    
except AttributeError as e:
    print(f"AttributeError: {e}")

except Exception as e:
    print(f"An error occurred: {e}")

Hello from module1
Variable from module2: I am from module2


# 2. Virtual Environment Management
Scenario: You are working on multiple Python projects with different dependencies and

versions. To avoid conflicts and ensure project isolation, you decide to use virtual environments.

Create a Python program that demonstrates the following:

1. Create a virtual environment for a specific project.

2. Activate and deactivate virtual environments,

3. Install, upgrade, and uninstall packages within a virtual environment.

4. List the installed packages in a virtual environment.

5. Implement error handling for virtual environment operations

In [ ]:
(base) C:\Users\jagad>pip install virtualenv

In [ ]:
(base) C:\Users\jagad>python -m venv venv_demo

In [ ]:
(base) C:\Users\jagad>cd venv_demo

In [ ]:
(base) C:\Users\jagad\venv_demo>cd Scripts

In [ ]:
(base) C:\Users\jagad\venv_demo\Scripts>activate

In [ ]:
(venv_demo) (base) C:\Users\jagad\venv_demo\Scripts>pip install numpy

In [ ]:
(venv_demo) (base) C:\Users\jagad\venv_demo\Scripts>deactivate

In [ ]:
(base) C:\Users\jagad\venv_demo\Scripts>

# 3. Module Dependency Resolution Scenario: You are developing a Python application that relies on third-party packages Managing

dependencies and ensuring compatibility is crucial for your project's success.

Design a Python program that demonstrates the following:

1. Use a requirements.txt file to specify project dependencies.
2. Automatically install all project dependencies from the requirements, txt    file
3. Ensure that the versions of installed packages are compatible
4. implement error handling for dependency resolution and installation

In [ ]:
(base) C:\Users\jagad>cd venv_demo

(base) C:\Users\jagad\venv_demo>cd Scripts

(base) C:\Users\jagad\venv_demo\Scripts>activate

In [ ]:
(venv_demo) (base) C:\Users\jagad\venv_demo>cd..

In [ ]:
(venv_demo) (base) C:\Users\jagad\venv_demo>pip freeze > requirements.txt

(venv_demo) (base) C:\Users\jagad\venv_demo>pip install -r requirements.txt
Collecting requests==2.26.0 (from -r requirements.txt (line 1))
  Downloading requests-2.26.0-py2.py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
Requirement already satisfied: numpy<2.0.0,>=1.21.0 in c:\users\jagad\venv_demo\lib\site-packages (from -r requirements.txt (line 2)) (1.25.2)
Collecting matplotlib<4.0.0,>=3.4.3 (from -r requirements.txt (line 3))
  Downloading matplotlib-3.8.0-cp311-cp311-win_amd64.whl (7.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 7.5 MB/s eta 0:00:00
Collecting flask<3.0.0,>=2.1.0 (from -r requirements.txt (line 4))
  Downloading flask-2.3.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 5.4 MB/s eta 0:00:00
Collecting pytest==6.2.5 (from -r requirements.txt (line 5))
  Downloading pytest-6.2.5-py3-none-any.whl (280 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.7/280.7 kB 8.7 MB/s eta 0:00:00
Collecting urllib3<1.27,>=1.21.1 (from requests==2.26.0->-r requirements.txt (line 1))
  Downloading urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 8.9 MB/s eta 0:00:00
Collecting certifi>=2017.4.17 (from requests==2.26.0->-r requirements.txt (line 1))
  Downloading certifi-2023.7.22-py3-none-any.whl (158 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 9.3 MB/s eta 0:00:00
Collecting charset-normalizer~=2.0.0 (from requests==2.26.0->-r requirements.txt (line 1))
  Downloading charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
Collecting idna<4,>=2.5 (from requests==2.26.0->-r requirements.txt (line 1))
  Downloading idna-3.4-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB ? eta 0:00:00
Collecting attrs>=19.2.0 (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading attrs-23.1.0-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB ? eta 0:00:00
Collecting iniconfig (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
Collecting packaging (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading packaging-23.1-py3-none-any.whl (48 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB ? eta 0:00:00
Collecting pluggy<2.0,>=0.12 (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading pluggy-1.3.0-py3-none-any.whl (18 kB)
Collecting py>=1.8.2 (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading py-1.11.0-py2.py3-none-any.whl (98 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB ? eta 0:00:00
Collecting toml (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading toml-0.10.2-py2.py3-none-any.whl (16 kB)
Collecting atomicwrites>=1.0 (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading atomicwrites-1.4.1.tar.gz (14 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Collecting colorama (from pytest==6.2.5->-r requirements.txt (line 5))
  Downloading colorama-0.4.6-py2.py3-none-any.whl (25 kB)
Collecting contourpy>=1.0.1 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading contourpy-1.1.0-cp311-cp311-win_amd64.whl (470 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.9/470.9 kB 7.3 MB/s eta 0:00:00
Collecting cycler>=0.10 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading cycler-0.11.0-py3-none-any.whl (6.4 kB)
Collecting fonttools>=4.22.0 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading fonttools-4.42.1-cp311-cp311-win_amd64.whl (2.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00
Collecting kiwisolver>=1.0.1 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading kiwisolver-1.4.5-cp311-cp311-win_amd64.whl (56 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.9 MB/s eta 0:00:00
Collecting pillow>=6.2.0 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading Pillow-10.0.1-cp311-cp311-win_amd64.whl (2.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.9 MB/s eta 0:00:00
Collecting pyparsing>=2.3.1 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading pyparsing-3.1.1-py3-none-any.whl (103 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.2 MB/s eta 0:00:00
Collecting python-dateutil>=2.7 (from matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 14.8 MB/s eta 0:00:00
Collecting Werkzeug>=2.3.7 (from flask<3.0.0,>=2.1.0->-r requirements.txt (line 4))
  Downloading werkzeug-2.3.7-py3-none-any.whl (242 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 4.9 MB/s eta 0:00:00
Collecting Jinja2>=3.1.2 (from flask<3.0.0,>=2.1.0->-r requirements.txt (line 4))
  Downloading Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB ? eta 0:00:00
Collecting itsdangerous>=2.1.2 (from flask<3.0.0,>=2.1.0->-r requirements.txt (line 4))
  Downloading itsdangerous-2.1.2-py3-none-any.whl (15 kB)
Collecting click>=8.1.3 (from flask<3.0.0,>=2.1.0->-r requirements.txt (line 4))
  Downloading click-8.1.7-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB ? eta 0:00:00
Collecting blinker>=1.6.2 (from flask<3.0.0,>=2.1.0->-r requirements.txt (line 4))
  Downloading blinker-1.6.2-py3-none-any.whl (13 kB)
Collecting MarkupSafe>=2.0 (from Jinja2>=3.1.2->flask<3.0.0,>=2.1.0->-r requirements.txt (line 4))
  Downloading MarkupSafe-2.1.3-cp311-cp311-win_amd64.whl (17 kB)
Collecting six>=1.5 (from python-dateutil>=2.7->matplotlib<4.0.0,>=3.4.3->-r requirements.txt (line 3))
  Downloading six-1.16.0-py2.py3-none-any.whl (11 kB)
Building wheels for collected packages: atomicwrites
  Building wheel for atomicwrites (pyproject.toml) ... done
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6973 sha256=7bddc226a418c0d70281f3fbe4eaf962697b8e5b4abf3bd4b301e32221970e14
  Stored in directory: c:\users\jagad\appdata\local\pip\cache\wheels\f7\99\9c\d24e98c35f30eba0c367ad1e7888d396d676abb35fe1e7611c
Successfully built atomicwrites
Installing collected packages: urllib3, toml, six, pyparsing, py, pluggy, pillow, packaging, MarkupSafe, kiwisolver, itsdangerous, iniconfig, idna, fonttools, cycler, contourpy, colorama, charset-normalizer, certifi, blinker, attrs, atomicwrites, Werkzeug, requests, python-dateutil, pytest, Jinja2, click, matplotlib, flask
Successfully installed Jinja2-3.1.2 MarkupSafe-2.1.3 Werkzeug-2.3.7 atomicwrites-1.4.1 attrs-23.1.0 blinker-1.6.2 certifi-2023.7.22 charset-normalizer-2.0.12 click-8.1.7 colorama-0.4.6 contourpy-1.1.0 cycler-0.11.0 flask-2.3.3 fonttools-4.42.1 idna-3.4 iniconfig-2.0.0 itsdangerous-2.1.2 kiwisolver-1.4.5 matplotlib-3.8.0 packaging-23.1 pillow-10.0.1 pluggy-1.3.0 py-1.11.0 pyparsing-3.1.1 pytest-6.2.5 python-dateutil-2.8.2 requests-2.26.0 six-1.16.0 toml-0.10.2 urllib3-1.26.16

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

(venv_demo) (base) C:\Users\jagad\venv_demo>